# use of API to download historical hourly data. INDICATIONS THAT A LIMIT EXISTS IN TERMS OF NUMBER OF DAYS ALLOWED TO DOWNLOAD WITH API...

In [8]:
# code based on input from website https://open-meteo.com/en/docs#latitude=51.6533&longitude=5.2875&current=&minutely_15=&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,cloud_cover,visibility,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_gusts_10m,soil_temperature_0cm,soil_temperature_6cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,direct_radiation&daily=&timezone=Europe%2FBerlin&past_days=92&past_minutely_15=4&forecast_hours=1&models=
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [14]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 51.6533,
	"longitude": 5.2875,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "cloud_cover", "visibility", "wind_speed_10m", "wind_speed_80m", "wind_speed_120m", "wind_speed_180m", "wind_gusts_10m", "soil_temperature_0cm", "soil_temperature_6cm", "soil_moisture_0_to_1cm", "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", "soil_moisture_9_to_27cm", "direct_radiation"],
	"timezone": "Europe/Berlin",
	"past_days": 7,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_visibility = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_80m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(10).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(11).ValuesAsNumpy()
hourly_soil_temperature_0cm = hourly.Variables(12).ValuesAsNumpy()
hourly_soil_temperature_6cm = hourly.Variables(13).ValuesAsNumpy()
hourly_soil_moisture_0_to_1cm = hourly.Variables(14).ValuesAsNumpy()
hourly_soil_moisture_1_to_3cm = hourly.Variables(15).ValuesAsNumpy()
hourly_soil_moisture_3_to_9cm = hourly.Variables(16).ValuesAsNumpy()
hourly_soil_moisture_9_to_27cm = hourly.Variables(17).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(18).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_temperature_0cm"] = hourly_soil_temperature_0cm
hourly_data["soil_temperature_6cm"] = hourly_soil_temperature_6cm
hourly_data["soil_moisture_0_to_1cm"] = hourly_soil_moisture_0_to_1cm
hourly_data["soil_moisture_1_to_3cm"] = hourly_soil_moisture_1_to_3cm
hourly_data["soil_moisture_3_to_9cm"] = hourly_soil_moisture_3_to_9cm
hourly_data["soil_moisture_9_to_27cm"] = hourly_soil_moisture_9_to_27cm
hourly_data["direct_radiation"] = hourly_direct_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 51.645999908447266°N 5.2779998779296875°E
Elevation 12.0 m asl
Timezone b'Europe/Berlin' b'GMT+1'
Timezone difference to GMT+0 3600 s
                         date  temperature_2m  relative_humidity_2m  \
0   2025-02-23 23:00:00+00:00       11.167500                  84.0   
1   2025-02-24 00:00:00+00:00       11.317500                  81.0   
2   2025-02-24 01:00:00+00:00       11.217500                  80.0   
3   2025-02-24 02:00:00+00:00       11.017500                  83.0   
4   2025-02-24 03:00:00+00:00       11.367499                  81.0   
..                        ...             ...                   ...   
331 2025-03-09 18:00:00+00:00       11.250000                  81.0   
332 2025-03-09 19:00:00+00:00       10.600000                  82.0   
333 2025-03-09 20:00:00+00:00       10.100000                  83.0   
334 2025-03-09 21:00:00+00:00        9.600000                  84.0   
335 2025-03-09 22:00:00+00:00        9.000000                  85.0   

 

In [15]:
# Filter the data for the last week
last_week_data = hourly_dataframe[hourly_dataframe['date'] >= (hourly_dataframe['date'].max() - pd.Timedelta(days=7))]


# Count the number of rows
row_count = len(last_week_data)
print(f"Number of rows: {row_count}")

Number of rows: 169


In [11]:
print(hourly_dataframe['date'].dtype)

datetime64[ns, UTC]


In [16]:
print(hourly_dataframe['date'].max())
print(hourly_dataframe['date'].min())

2025-03-09 22:00:00+00:00
2025-02-23 23:00:00+00:00


In [18]:
last_week_data.to_csv('last_week_hourly_data.csv', index=False)